In [3]:
# importing libraries
from tkinter import *
from tkinter import font
import tkinter as tk
from PIL import Image, ImageTk
import os
import subprocess
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import time
import csv
from datetime import datetime

def open_file():

    mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) # keep_all=False
    mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) # keep_all=True
    resnet = InceptionResnetV1(pretrained='vggface2').eval() 

    dataset=datasets.ImageFolder('C:\\Users\\hp\\Desktop\\minor_project\\minor_code_files\\data') # photos folder path 
    idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

    def collate_fn(x):
        return x[0]

    loader = DataLoader(dataset, collate_fn=collate_fn)

    name_list = [] # list of names corrospoing to cropped photos
    embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

    for img, idx in loader:
        face, prob = mtcnn0(img, return_prob=True) 
        if face is not None and prob>0.92:
            emb = resnet(face.unsqueeze(0)) 
            embedding_list.append(emb.detach()) 
            name_list.append(idx_to_class[idx])        

    # save data
    data = [embedding_list, name_list] 
    torch.save(data, 'data.pt') # saving data.pt file

    # loading data.pt file
    load_data = torch.load('data.pt') 
    embedding_list = load_data[0] 
    name_list = load_data[1] 

    # Using webcam recognize face

    # loading data.pt file
    load_data = torch.load('data.pt') 
    embedding_list = load_data[0] 
    name_list = load_data[1] 

    cam = cv2.VideoCapture(0) 
    written_names = []

    while True:
        ret, frame = cam.read()
        if not ret:
            print("fail to grab frame, try again")
            break

        img = Image.fromarray(frame)
        img_cropped_list, prob_list = mtcnn(img, return_prob=True) 

        if img_cropped_list is not None:
            boxes, _ = mtcnn.detect(img)

            for i, prob in enumerate(prob_list):
                if prob>0.90:
                    emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 

                    dist_list = [] # list of matched distances, minimum distance is used to identify the person

                    for idx, emb_db in enumerate(embedding_list):
                        dist = torch.dist(emb, emb_db).item()
                        dist_list.append(dist)

                    min_dist = min(dist_list) # get minumum dist value
                    min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                    name = name_list[min_dist_idx] # get name corrosponding to minimum dist

                    box = boxes[i] 

                    original_frame = frame.copy() # storing copy of frame before drawing on it

                    if min_dist<0.90 and name not in written_names:
                        frame = cv2.putText(frame, name+' '+str(min_dist), (int(box[0]),int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA)
                        data_dict = {"Name": [name], "Time": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")]}

                        # Write the data to a CSV file
                        with open('C:\\Users\\hp\\Desktop\\minor_project\\minor_code_files\\recog.csv', mode='a') as file:
                            writer = csv.writer(file)
                            if file.tell() == 0:  # Check if file is empty
                                writer.writerow(["Name", "Time"])  # Write header row
                            writer.writerow([name, datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
                        written_names.append(name)


                    frame = cv2.rectangle(frame, (int(box[0]),int(box[1])) , (int(box[2]),int(box[3])), (255,0,0), 2)

        cv2.imshow("IMG", frame)


        k = cv2.waitKey(1)
        if k%256==27: # ESC
            print('Esc pressed, closing...')
            break

        elif k%256==32: # space to save image
            print('Enter your name :')
            name = input()

            # create directory if not exists
            if not os.path.exists(f'C:/Users/hp/Desktop/Face/data/{name}'):
                os.mkdir(f'C:/Users/hp/Desktop/Face/data/{name}')

            img_name = "C:/Users/hp/Desktop/Face/data/{}/{}.jpg".format(name, int(time.time()))
            cv2.imwrite(img_name, original_frame)
            print(" saved: {}".format(img_name))


    cam.release()
    cv2.destroyAllWindows()
 
 
#Create window
root = tk.Tk()
 
#window title
root.title("Khwopa College of Engeneering")
 
#Create topframe with top side
topframe=Frame(root)
my_font=tk.font.Font(family="Rekha",size=30,weight="bold",slant="italic")
#create label on root window
label=Label(root,text="Khwopa College of Engeneering Attendance System",font=my_font,foreground="#283142").pack()
#pack topframe on top side
topframe.pack(side=TOP)
 
# Create a frame for the image
image_frame = tk.Frame(root)
image_frame.pack(side=tk.LEFT, padx=10, pady=10)
 
# Load the image using PIL
#image = Image.open("C:\\Users\\hp\\Desktop\\routine.jpg")
 
# Create a PhotoImage object from the image
#photo = ImageTk.PhotoImage(image)
 
# Add the image to the frame
#canvas = tk.Canvas(image_frame, width=photo.width(), height=photo.height())
#canvas.create_image(0, 0, anchor='nw', image=photo)
#canvas.pack()
 
# Create a frame for the button
button_frame = tk.Frame(root)
button_frame.pack(side=tk.RIGHT, padx=10)
 
# Add the Start button to the frame
start_button = tk.Button(button_frame, text="Start",width=8, height=3,bg="black",fg="white", font=("Arial", 24),command=open_file)
start_button.pack(pady=10, padx=(0, 200))
 
root.mainloop()


Esc pressed, closing...
